<a href="https://colab.research.google.com/github/smslca/Huggigfaces/blob/main/HuggingFaces_tutorial_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install transformers accelerate datasets evaluate

In [9]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer,DataCollatorWithPadding,TrainingArguments,Trainer
from datasets import load_dataset
import numpy as np
import evaluate

In [3]:
raw_dataset = load_dataset('glue','mrpc')

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [4]:
checkpoint = 'distilbert-base-uncased-finetuned-sst-2-english'
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [5]:
def tokenizer_function(example):
  return tokenizer(example['sentence1'],example['sentence2'],truncation=True)

tokenized_dataset = raw_dataset.map(tokenizer_function,batched=True)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [10]:
def compute_metrics(eval_preds):
  logits,labels = eval_preds
  preds = np.argmax(logits,axis=-1)
  metrics = evaluate.load('glue','mrpc')
  return metrics.compute(predictions=preds,references=labels)

In [7]:
data_collator = DataCollatorWithPadding(tokenizer)

In [14]:
training_args = TrainingArguments('test_trainer',eval_strategy='epoch',report_to='none')

In [15]:
trainer = Trainer(model,training_args,train_dataset=tokenized_dataset['train'],eval_dataset=tokenized_dataset['validation'],
                  data_collator=data_collator,tokenizer=tokenizer,compute_metrics=compute_metrics)

<ipython-input-15-ed949d2d9311>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model,training_args,train_dataset=tokenized_dataset['train'],eval_dataset=tokenized_dataset['validation'],


In [16]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.450623,0.818627,0.879085
2,0.528500,0.535327,0.833333,0.884354
3,0.294500,0.815250,0.833333,0.885906


TrainOutput(global_step=1377, training_loss=0.340142310314968, metrics={'train_runtime': 132.3959, 'train_samples_per_second': 83.114, 'train_steps_per_second': 10.401, 'total_flos': 203961502572816.0, 'train_loss': 0.340142310314968, 'epoch': 3.0})